In [22]:
#change as necessary
LOCALROOT = '/Users/bellahwang/Documents/GitHub/'

# Querying for Specialized Verb Forms
This script uses the ElementTree package to parse the xml of Homer's 'Odyssey' and to produce the number of ἄν modified subjunctives and to print them out.

In [16]:
import xml.etree.ElementTree as ET
import os

FILENAME = 'tlg0012.tlg002.perseus-grc1.tb.xml'
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)

tree = ET.parse(LOCALPATH)
root = tree.getroot()

count  = 1    #sets counter keeping track of query results
subjid = -10  #in case the code encounters a subjunctive first
anhead = -20  #in case the code encounters an ἄν first

for body in root.findall('./body'): 
    for sentence in body.findall('./sentence'):
        for word in sentence.findall('./word'):
            if 'postag' in word.attrib:
                postag = word.get('postag')
                mood   = postag[4] #narrows down to verb mood
                lemma  = word.get('lemma')
                if lemma == 'ἄν':
                    anhead = word.get('head')
                    anword = word.attrib
                elif mood == 's': 
                    subjid = word.get('id')
                    subjword = word.attrib
                if anhead == subjid: #checks if ἄν modifies subjunctive
                    print(sentence.attrib)
                    print('\t', anword)
                    print('\t', subjword)
                    
                    count += 1
                    
                    subjid = -10  
                    anhead = -20

1
{'subdoc': '1.81-1.87', 'id': '2185583', 'document_id': 'urn:cts:greekLit:tlg0012.tlg002.perseus-grc1'}
	 {'id': '48', 'form': 'κε', 'lemma': 'ἄν', 'postag': 'g--------', 'head': '49', 'relation': 'AuxZ', 'cite': 'urn:cts:greekLit:tlg0012.tlg002:1.87'}
	 {'id': '49', 'form': 'νέηται', 'lemma': 'νέομαι', 'postag': 'v3spse---', 'head': '47', 'relation': 'ADV', 'cite': 'urn:cts:greekLit:tlg0012.tlg002:1.87'}
2
{'subdoc': '1.158', 'id': '2185616', 'document_id': 'urn:cts:greekLit:tlg0012.tlg002.perseus-grc1'}
	 {'id': '10', 'form': 'κεν', 'lemma': 'ἄν', 'postag': 'g--------', 'head': '11', 'relation': 'AuxZ', 'cite': 'urn:cts:greekLit:tlg0012.tlg002:1.158'}
	 {'id': '11', 'form': 'εἴπω', 'lemma': 'εἶπον', 'postag': 'v1sasa---', 'head': '8', 'relation': 'OBJ', 'cite': 'urn:cts:greekLit:tlg0012.tlg002:1.158'}
3
{'subdoc': '1.187-1.193', 'id': '2185634', 'document_id': 'urn:cts:greekLit:tlg0012.tlg002.perseus-grc1'}
	 {'id': '44', 'form': 'ἄν', 'lemma': 'ἄν', 'postag': 'g--------', 'head': 

# Sentence Alignment

In [7]:
import sys
import os
import xml.etree.ElementTree as ET

FILENAME   = "atmurray.odyssey-sentalign.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'pharrqueries', 'queryscripts', FILENAME)

Engtree = ET.parse(LOCALPATH)
Engroot = Engtree.getroot()

def printEngSent(target):
    for body in Engroot.findall('.//{http://www.tei-c.org/ns/1.0}body'):
        for translation in body.findall('./{http://www.tei-c.org/ns/1.0}div'):
            for book in translation.findall('./{http://www.tei-c.org/ns/1.0}div'):
                for card in book.findall('./{http://www.tei-c.org/ns/1.0}div'):
                    for p in card.findall('./{http://www.tei-c.org/ns/1.0}p'):
                        for s in p.findall('./{http://www.tei-c.org/ns/1.0}s'):
                            id = s.get('{http://www.w3.org/XML/1998/namespace}id')
                            #print(id) #for debugging the english xml
                            if (target == id[4:11]):
                                sys.stdout.write(s.text)

In [8]:
# change FILENAME to local path
FILENAME = "tlg0012.tlg002.perseus-grc1.tb.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)

Grktree = ET.parse(LOCALPATH)
Grkroot = Grktree.getroot()

def printGrkSent(targetid, givenid):
    for sentence in Grkroot.findall('.//sentence'): 
        sentid = sentence.get('id')
        if (sentid == targetid):
            for word in sentence.findall('./word'):
                wordid = word.get('id')
                form = word.get('form')
                lemma = word.get('lemma')
                postag = word.get('postag')
                if (postag != None):
                    if (wordid == givenid):

                        # comment out if converting to html
                                        #  bold        green               end
                        sys.stdout.write('\033[1m' + '\33[32m' + form + '\033[0m')
                        sys.stdout.write(" ")

                        # for html file conversion
                        # sys.stdout.write('<b>' + form + '</b>')
                        # sys.stdout.write(" ")

                        continue
                    sys.stdout.write(form)
                    sys.stdout.write(" ")

def printGrkSent2(targetid, firstid, secondid):
    for sentence in Grkroot.findall('.//sentence'): 
        sentid = sentence.get('id')
        if (sentid == targetid):
            for word in sentence.findall('./word'):
                wordid = word.get('id')
                form = word.get('form')
                lemma = word.get('lemma')
                postag = word.get('postag')
                if (postag != None):
                    if (wordid == firstid or wordid == secondid):

                        # comment out if converting to html
                                        #  bold        green               end
                        sys.stdout.write('\033[1m' + '\33[32m' + form + '\033[0m')
                        sys.stdout.write(" ")
                        # printVocabSection(lemma)

                        # for html file conversion
                        # sys.stdout.write('<b>' + form + '</b>')
                        # sys.stdout.write(" ")

                        continue
                    sys.stdout.write(form)
                    sys.stdout.write(" ")

In [9]:
# change FILENAME to local path
FILENAME = "tlg0012.tlg002.perseus-grc1.tb.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)
tree = ET.parse(LOCALPATH)
root = tree.getroot()

count = 1           # sets counter keeping track of query results
subjid = -10       # in case the code encounters a subjunctive first
anhead = -20       # in case the code encounters an ἄν first
anid = -30

# for html file conversion
# sys.stdout.write("<table>" + "\n")

for sentence in root.findall('.//sentence'):
    sentid = sentence.get('id')
    for word in sentence.findall('./word'):
        if 'postag' in word.attrib:
            postag = word.get('postag')
            mood = postag[4]
            form = word.get('form')
            cite = word.get('cite')
            lemma = word.get('lemma')
            if lemma == 'ἄν':
                anhead = word.get('head')
                anid = word.get('id')
            else: #only other possibility is if the word is a subjunctive
                subjid = word.get('id')
                if (mood == 'o'):
                    if anhead == subjid: #checks if ἄν modifies subjunctive

                        # comment out if converting to html
                        print(count, "form: ", form, "lemma: ", lemma, "cite: ", cite)
                        printGrkSent2(sentid, subjid, anid)
                        print('')
                        printEngSent(sentid)
                        print('')
                        print('')
                        count += 1
                        
                        subjid = -10  
                        anhead = -20
                        anid = -30
                        
                        # for html file conversion
                        # sys.stdout.write("\t" + "<tr>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>" + form + "</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>" + lemma + "</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>" + cite + "</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>")
                        # printGrkSent2(sentid, subjid, anid)
                        # sys.stdout.write("</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>")
                        # printEngSent(sentid)
                        # sys.stdout.write("\t" + "\t" + "</td>" + "\n")
                        # sys.stdout.write("\t" + "</tr>" + "\n")

                    else:

                        # comment out if converting to html
                        print(count, "form: ", form, "lemma: ", lemma, "cite: ", cite)
                        printGrkSent(sentid, subjid)
                        print('')
                        printEngSent(sentid)
                        print('')
                        print('')
                        count += 1
                        
                        subjid = -10  
                        anhead = -20
                        anid = -30
                        
                        # for html file conversion
                        # sys.stdout.write("\t" + "<tr>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>" + form + "</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>" + lemma + "</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>" + cite + "</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>")
                        # printGrkSent(sentid, subjid)
                        # sys.stdout.write("</td>" + "\n")
                        # sys.stdout.write("\t" + "\t" + "<td>")
                        # printEngSent(sentid)
                        # sys.stdout.write("\t" + "\t" + "</td>" + "\n")
                        # sys.stdout.write("\t" + "</tr>" + "\n")

# for html file conversion
# sys.stdout.write("</table>")


1 form:  ἀπόλοιτο lemma:  ἀπόλλυμι cite:  urn:cts:greekLit:tlg0012.tlg002:1.47
ὡς ἀπόλοιτο καὶ ἄλλος , ὅτις τοιαῦτά γε ῥέζοι · 
so, too, may any other also be destroyed who does such deeds. 

2 form:  ῥέζοι lemma:  ῥέζω cite:  urn:cts:greekLit:tlg0012.tlg002:1.47
ὡς ἀπόλοιτο καὶ ἄλλος , ὅτις τοιαῦτά γε ῥέζοι · 
so, too, may any other also be destroyed who does such deeds. 

3 form:  λαθοίμην lemma:  λανθάνω cite:  urn:cts:greekLit:tlg0012.tlg002:1.65
πῶς ἂν ἔπειτ' Ὀδυσῆος ἐγὼ θείοιο λαθοίμην , ὃς περὶ μὲν νόον ἐστὶ βροτῶν , περὶ δ' ἱρὰ θεοῖσιν ἀθανάτοισιν ἔδωκε , τοὶ οὐρανὸν εὐρὺν ἔχουσιν ; 
How should I, then, forget godlike Odysseus, who is beyond all mortals in wisdom, and beyond all has paid sacrifice to the immortal gods, who hold broad heaven? 

4 form:  θείη lemma:  τίθημι cite:  urn:cts:greekLit:tlg0012.tlg002:1.116
τὴν δὲ πολὺ πρῶτος ἴδε Τηλέμαχος θεοειδής , ἧστο γὰρ ἐν μνηστῆρσι φίλον τετιημένος ἦτορ , ὀσσόμενος πατέρ' ἐσθλὸν ἐνὶ φρεσίν , εἴ ποθεν ἐλθὼν μνηστήρων τῶν μὲν σκέδ

then should they all find swift destruction and bitterness in their wooing. 

20 form:  τλαίης lemma:  τλάω cite:  urn:cts:greekLit:tlg0012.tlg002:1.288
εἰ μέν κεν πατρὸς βίοτον καὶ νόστον ἀκούσῃς , ἦ τ' ἂν τρυχόμενός περ ἔτι τλαίης ἐνιαυτόν · 
If so be thou shalt hear that thy father is alive and coming home, then verily, though thou art sore afflicted, thou couldst endure for yet a year. 

21 form:  ὄλοισθε lemma:  ὄλλυμι cite:  urn:cts:greekLit:tlg0012.tlg002:1.380
νήποινοί κεν ἔπειτα δόμων ἔντοσθεν ὄλοισθε . " 
Without atonement, then, should ye perish within my halls. 

22 form:  ποιήσειεν lemma:  ποιέω cite:  urn:cts:greekLit:tlg0012.tlg002:1.387
μὴ σέ γ' ἐν ἀμφιάλῳ Ἰθάκῃ βασιλῆα Κρονίων ποιήσειεν , ὅ τοι γενεῇ πατρώιόν ἐστιν . " 
May the son of Cronos never make thee king in sea-girt Ithaca, which thing is by birth thy heritage. 

23 form:  ἐθέλοιμι lemma:  ἐθέλω cite:  urn:cts:greekLit:tlg0012.tlg002:1.390
καὶ κεν τοῦτ' ἐθέλοιμι Διός γε διδόντος ἀρέσθαι . 
Even this should I be

For not yet have I come near to the shore of Achaea, nor have I as yet set foot on my own land, but have ever been wandering, laden with woe, from the day when first I went with goodly Agamemnon to Ilios, famed for its horses, to fight with the Trojans. 

54 form:  ἐρέοιμι lemma:  ἔρομαι cite:  urn:cts:greekLit:tlg0012.tlg002:11.229
αἱ δ' ἀμφ' αἷμα κελαινὸν ἀολλέες ἠγερέθοντο , αὐτὰρ ἐγὼ βούλευον ὅπως ἐρέοιμι ἑκάστην . 
These flocked in throngs about the dark blood, and I considered how I might question each; 

55 form:  ἐλάσειε lemma:  ἐλαύνω cite:  urn:cts:greekLit:tlg0012.tlg002:11.290
οὐ δ' ἄρα Νηλεὺς τῷ ἐδίδου ὃς μὴ ἕλικας βόας εὐρυμετώπους ἐκ Φυλάκης ἐλάσειε βίης Ἰφικληείης ἀργαλέας · 
but Neleus would give her to no man, save to him who should drive from Phylace the kine of mighty Iphicles, sleek and broad of brow; and hard they were to drive. 

56 form:  εἴη lemma:  εἰμί cite:  urn:cts:greekLit:tlg0012.tlg002:11.316
Ὄσσαν ἐπ' Οὐλύμπῳ μέμασαν θέμεν , αὐτὰρ ἐπ' Ὄσσῃ Πήλιον εἰνοσί

but as often as he was about to heave it over the top, the weight would turn it back, 

79 form:  τεχνήσαιτο lemma:  τεχνάομαι cite:  urn:cts:greekLit:tlg0012.tlg002:11.613
μὴ τεχνησάμενος μη δ' ἄλλο τι τεχνήσαιτο , ὃς κεῖνον τελαμῶνα ἑῇ ἐγκάτθετο τέχνῃ . 
May he never have designed, or hereafter design such another, even he who stored up in his craft the device of that belt. 

80 form:  ἔλθοι lemma:  ἔρχομαι cite:  urn:cts:greekLit:tlg0012.tlg002:11.628
' " ὣς εἰπὼν ὁ μὲν αὖτις ἔβη δόμον Ἄϊδος εἴσω , αὐτὰρ ἐγὼν αὐτοῦ μένον ἔμπεδον , εἴ τις ἔτ' ἔλθοι ἀνδρῶν ἡρώων , οἳ δὴ τὸ πρόσθεν ὄλοντο . 
So saying, he went his way again into the house of Hades, but I abode there steadfastly, in the hope that some other haply might still come forth of the warrior heroes who died in the days of old. 

81 form:  πέμψειεν lemma:  πέμπω cite:  urn:cts:greekLit:tlg0012.tlg002:11.635
ἐμὲ δὲ χλωρὸν δέος ᾕρει , μή μοι Γοργείην κεφαλὴν δεινοῖο πελώρου ἐξ Ἀίδεω πέμψειεν ἀγαυὴ Περσεφόνεια . 
and pale fear seiz

And the strong and mighty Alcinous went himself throughout the ship, and carefully stowed the gifts beneath the benches, that they might not hinder any of the crew at their rowing, when they busily plied the oars. 

113 form:  σπερχοίατ' lemma:  σπέρχω cite:  urn:cts:greekLit:tlg0012.tlg002:13.22
καὶ τὰ μὲν εὖ κατέθηχ' ἱερὸν μένος Ἀλκινόοιο , αὐτὸς ἰὼν διὰ νηὸς ὑπὸ ζυγά , μή τιν' ἑταίρων βλάπτοι ἐλαυνόντων , ὁπότε σπερχοίατ' ἐρετμοῖς . 
And the strong and mighty Alcinous went himself throughout the ship, and carefully stowed the gifts beneath the benches, that they might not hinder any of the crew at their rowing, when they busily plied the oars. 

114 form:  ποιήσειαν lemma:  ποιέω cite:  urn:cts:greekLit:tlg0012.tlg002:13.42
ἤδη γὰρ τετέλεσται ἅ μοι φίλος ἤθελε θυμός , πομπὴ καὶ φίλα δῶρα , τά μοι θεοὶ Οὐρανίωνες ὄλβια ποιήσειαν · 
For now all that my heart desired has been brought to pass: a convoy, and gifts of friendship. 

115 form:  εὕροιμι lemma:  εὑρίσκω cite:  urn:cts:greekLi

...

αὐτὰρ ἐγὼ σῖτον καὶ ὕδωρ καὶ οἶνον ἐρυθρὸν ἐνθήσω μενοεικέ' , ἅ κέν τοι λιμὸν ἐρύκοι , εἵματά τ' ἀμφιέσω · 
And I will place therein bread and water and red wine to satisfy thy heart, to keep hunger from thee. And I will clothe thee with raiment. 

618 form:  ἐπιβαίην lemma:  ἐπιβαίνω cite:  urn:cts:greekLit:tlg0012.tlg002:5.177
οὐ δ' ἂν ἐγὼν ἀέκητι σέθεν σχεδίης ἐπιβαίην , εἰ μή μοι τλαίης γε , θεά , μέγαν ὅρκον ὀμόσσαι μή τί μοι αὐτῷ πῆμα κακὸν βουλευσέμεν ἄλλο . " 
But I will not set foot on a raft in thy despite, unless thou, goddess, wilt bring thyself to swear a mighty oath that thou wilt not plot against me any fresh mischief to my hurt. 

619 form:  τλαίης lemma:  τλάω cite:  urn:cts:greekLit:tlg0012.tlg002:5.178
οὐ δ' ἂν ἐγὼν ἀέκητι σέθεν σχεδίης ἐπιβαίην , εἰ μή μοι τλαίης γε , θεά , μέγαν ὅρκον ὀμόσσαι μή τί μοι αὐτῷ πῆμα κακὸν βουλευσέμεν ἄλλο . " 
But I will not set foot on a raft in thy despite, unless thou, goddess, wilt bring thyself to swear a mighty oath that thou wil

and Odysseus pondered whether he should clasp the knees of the fair-faced maid, and make his prayer, or whether, standing apart as he was, he should beseech her with gentle words, in hope that she might show him the city and give him raiment. 

641 form:  λίσσοιτ' lemma:  λίσσομαι cite:  urn:cts:greekLit:tlg0012.tlg002:6.144
ὁ δὲ μερμήριξεν Ὀδυσσεύς , ἢ γούνων λίσσοιτο λαβὼν ἐυώπιδα κούρην , ἦ αὔτως ἐπέεσσιν ἀποσταδὰ μειλιχίοισι λίσσοιτ' , εἰ δείξειε πόλιν καὶ εἵματα δοίη . 
and Odysseus pondered whether he should clasp the knees of the fair-faced maid, and make his prayer, or whether, standing apart as he was, he should beseech her with gentle words, in hope that she might show him the city and give him raiment. 

642 form:  δείξειε lemma:  δείκνυμι cite:  urn:cts:greekLit:tlg0012.tlg002:6.144
ὁ δὲ μερμήριξεν Ὀδυσσεύς , ἢ γούνων λίσσοιτο λαβὼν ἐυώπιδα κούρην , ἦ αὔτως ἐπέεσσιν ἀποσταδὰ μειλιχίοισι λίσσοιτ' , εἰ δείξειε πόλιν καὶ εἵματα δοίη . 
and Odysseus pondered whether he should c

ἰδόντα με καὶ λίποι αἰὼν κτῆσιν ἐμήν , δμῶάς τε καὶ ὑψερεφὲς μέγα δῶμα . " 
Yea, let life leave me, when I have seen once more my possessions, my slaves, and my great high-roofed house. 

664 form:  ἔλποιο lemma:  ἔλπω cite:  urn:cts:greekLit:tlg0012.tlg002:7.293
ἡ δ' οὔ τι νοήματος ἤμβροτεν ἐσθλοῦ , ὡς οὐκ ἂν ἔλποιο νεώτερον ἀντιάσαντα ἐρξέμεν · 
and she in no wise failed in good understanding, to do as thou wouldst not deem that one of younger years would do on meeting thee; 

665 form:  ἐπισκύσσαιτο lemma:  ἐπισκύζομαι cite:  urn:cts:greekLit:tlg0012.tlg002:7.306
ἡ μὲν γάρ μ' ἐκέλευε σὺν ἀμφιπόλοισιν ἕπεσθαι , ἀλλ' ἐγὼ οὐκ ἔθελον δείσας αἰσχυνόμενός τε , μή πως καὶ σοὶ θυμὸς ἐπισκύσσαιτο ἰδόντι · 
She did indeed bid me follow with her maidens, but I would not for fear and shame, lest haply thy heart should darken with wrath as thou sawest it; 

666 form:  δοίην lemma:  δίδωμι cite:  urn:cts:greekLit:tlg0012.tlg002:7.314
οἶκον δέ κ' ἐγὼ καὶ κτήματα δοίην , εἴ κ' ἐθέλων γε μένοις · 
a

πῶς ἂν ἐγώ σε δέοιμι μετ' ἀθανάτοισι θεοῖσιν , εἴ κεν Ἄρης οἴχοιτο χρέος καὶ δεσμὸν ἀλύξας ;" 
How could I put thee in bonds among the immortal gods, if Ares should avoid both the debt and the bonds and depart? 

698 form:  φέροιεν lemma:  φέρω cite:  urn:cts:greekLit:tlg0012.tlg002:8.409
ἔπος δ' εἴ πέρ τι βέβακται δεινόν , ἄφαρ τὸ φέροιεν ἀναρπάξασαι ἄελλαι . 
but if any word has been spoken that was harsh, may the storm-winds straightway snatch it and bear it away. 

699 form:  δοῖεν lemma:  δίδωμι cite:  urn:cts:greekLit:tlg0012.tlg002:8.411
σοὶ δὲ θεοὶ ἄλοχόν τ' ἰδέειν καὶ πατρίδ' ἱκέσθαι δοῖεν , ἐπεὶ δὴ δηθὰ φίλων ἄπο πήματα πάσχεις . " 
And for thyself, may the gods grant thee to see thy wife, and to come to thy native land, for long time hast thou been suffering woes far from thy friends. 

700 form:  δοῖεν lemma:  δίδωμι cite:  urn:cts:greekLit:tlg0012.tlg002:8.413
" καὶ σὺ φίλος μάλα χαῖρε , θεοὶ δέ τοι ὄλβια δοῖεν . 
All hail to thee, too, friend; and may the gods grant thee 

αὐτὰρ ἐγὼ λιπόμην κακὰ βυσσοδομεύων , εἴ πως τισαίμην , δοίη δέ μοι εὖχος Ἀθήνη . 
and I was left there, devising evil in the deep of my heart, if in any way I might take vengeance on him, and Athena grant me glory. 

728 form:  φοροίη lemma:  φορέω cite:  urn:cts:greekLit:tlg0012.tlg002:9.320
τὸ μὲν ἔκταμεν , ὄφρα φοροίη αὐανθέν . 
which he had cut to carry with him when dry; 

729 form:  τολμήσειεν lemma:  τολμάω cite:  urn:cts:greekLit:tlg0012.tlg002:9.332
αὐτὰρ τοὺς ἄλλους κλήρῳ πεπαλάσθαι ἄνωγον , ὅς τις τολμήσειεν ἐμοὶ σὺν μοχλὸν ἀείρας τρῖψαι ἐν ὀφθαλμῷ , ὅτε τὸν γλυκὺς ὕπνος ἱκάνοι . 
And I bade my comrades cast lots among them, which of them should have the hardihood with me to lift the stake and grind it into his eye when sweet sleep should come upon him. 

730 form:  ἱκάνοι lemma:  ἱκάνω cite:  urn:cts:greekLit:tlg0012.tlg002:9.333
αὐτὰρ τοὺς ἄλλους κλήρῳ πεπαλάσθαι ἄνωγον , ὅς τις τολμήσειεν ἐμοὶ σὺν μοχλὸν ἀείρας τρῖψαι ἐν ὀφθαλμῷ , ὅτε τὸν γλυκὺς ὕπνος ἱκάνοι . 
And I bad

In [26]:
import xml.etree.ElementTree as ET
import os

FILENAME   = "tlg0012.tlg002.perseus-grc1.tb.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)

tree = ET.parse(LOCALPATH)
root = tree.getroot()

count = 1

print("Please input the lemma of a vocabulary word for all known examples of the word in the Odyssey.")
vocab = input("Input lemma: ")

for sentence in root.findall('.//sentence'):
    subdoc = sentence.get('subdoc')
    sentid = sentence.get('id')
    for word in sentence.findall('./word'):
        wordid = word.get('id')
        form = word.get('form')
        lemma = word.get('lemma')
        cite = word.get('cite')
        if (vocab == lemma):
            print(count, "form: ", form, "lemma: ", lemma, "cite: ", cite)
            printGrkSent(sentid, wordid)
            print('')
            printEngSent(sentid)
            print('')
            print('')
            count += 1

Please input the lemma of a vocabulary word for all known examples of the word in the Iliad.
Input lemma: μῆνις
1 form:  μῆνιν lemma:  μῆνις cite:  urn:cts:greekLit:tlg0012.tlg002:14.283
ἦ μέν μοι μάλα πολλοὶ ἐπήϊσσον μελίῃσιν , ἱέμενοι κτεῖναι - δὴ γὰρ κεχολώατο λίην - ἀλλ' ἀπὸ κεῖνος ἔρυκε , Διὸς δ' ὠπίζετο μῆνιν ξεινίου , ὅς τε μάλιστα νεμεσσᾶται κακὰ ἔργα . 
Verily full many rushed upon me with their ashen spears, eager to slay me, for they were exceeding angry. But he warded them off, and had regard for the wrath of Zeus, the stranger's god, who above all others hath indignation at evil deeds. 

2 form:  μῆνιν lemma:  μῆνις cite:  urn:cts:greekLit:tlg0012.tlg002:2.66
θεῶν δ' ὑποδείσατε μῆνιν , μή τι μεταστρέψωσιν ἀγασσάμενοι κακὰ ἔργα . 
and fear the wrath of the gods, lest haply they turn against you in anger at your evil deeds. 

3 form:  μήνιος lemma:  μῆνις cite:  urn:cts:greekLit:tlg0012.tlg002:3.135
τῶ σφεων πολέες κακὸν οἶτον ἐπέσπον μήνιος ἐξ ὀλοῆς γλαυκώπιδος ὀβριμοπάτρης

# Adjacency List & Network Creation

In [28]:
import xml.etree.ElementTree as ET
import networkx as nx
import pandas as pd

# change FILENAME to local path
FILENAME = "tlg0012.tlg001.perseus-grc1.tb.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)
tree = ET.parse(LOCALPATH)
root = tree.getroot()

count = 1         #sets counter keeping track of query results
verbid = -10    #in case the code encounters a verb first
prephead = -20  #in case the code encounters a prep first
LEMMA = 'ὑπό'  #change as needed
verblemma = 'tester'
prepdict = {}

for body in root.findall('./body'): 
    for sentence in body.findall('./sentence'):
        for word in sentence.findall('./word'):
            if 'postag' in word.attrib:
                postag = word.get('postag')
                pos = postag[0]
                
                if (pos == 'r'):
                    preplemma = word.get('lemma')
                    if preplemma == LEMMA:
                        prephead = word.get('head')
                        if verbid == prephead:
                            objcount = prepdict.get(verblemma)
                            if (objcount == None):
                                prepdict.update({verblemma: 1})
                            else:
                                objcount += 1
                                prepdict.update({verblemma: objcount})
                        #print(word.attrib)
                elif (pos == 'v'):
                    verbid = word.get('id')
                    verblemma = word.get('lemma')
                    if verbid == prephead:
                        objcount = prepdict.get(verblemma)
                        if (objcount == None):
                            prepdict.update({verblemma: 1})
                        else:
                            objcount += 1
                            prepdict.update({verblemma: objcount})
                            #print(word.attrib)
                            #print(count) #counts query results
                            #count+=1
        verbid = -10    
        prephead = -20

#list1 = prepdict.values()
#print(sum(list1))

sortedDict = sorted(prepdict.items(), key=lambda x: x[1], reverse=True)
df = pd.DataFrame(sortedDict)
#df.to_csv('prepdeps.csv', index=False)

print(*sortedDict, sep = '\n')

('δαμάζω', 34)
('βάλλω', 12)
('λύω', 11)
('ὄρνυμι', 8)
('αἱρέω', 8)
('τίκτω', 8)
('φεύγω', 7)
('ἄγω', 6)
('ἔρχομαι', 6)
('δέω', 5)
('πίπτω', 5)
('εἶμι', 5)
('γίγνομαι', 4)
('τύπτω', 4)
('ὄλλυμι', 4)
('ἔχω', 3)
('λαμβάνω', 3)
('χάζω', 3)
('φοβέω', 3)
('χωρέω', 3)
('ἐρύω1', 3)
('ζεύγνυμι', 3)
('τρέμω', 2)
('πάσχω', 2)
('δάμνημι', 2)
('νύσσω', 2)
('ἐλύω', 2)
('εἰμί', 2)
('ἀλύσκω', 2)
('τιτύσκομαι', 2)
('ἁρπάζω', 2)
('ἐλαύνω', 2)
('ἐκπεράω', 2)
('φθίω', 2)
('ἁλίσκομαι', 2)
('κοιμάω', 2)
('θείνω', 2)
('ἐρύω2', 2)
('εἰσαναβαίνω', 2)
('τίθημι', 2)
('ῥώομαι', 2)
('στεναχίζω', 2)
('ναίω', 2)
('πτώσσω', 2)
('κεῖμαι', 2)
('κλονέω', 2)
('ἕλκω', 2)
('τείνω', 2)
('ἵστημι', 2)
('εἶδον', 2)
('γυιόω', 2)
('τελέω', 2)
('τρομέω', 1)
('στορέννυμι', 1)
('τανύω', 1)
('ἱκάνω', 1)
('φέβομαι', 1)
('ἐκπίπτω', 1)
('ἱκνέομαι', 1)
('οὐτάω', 1)
('ἀναπίμπλημι', 1)
('πέλω', 1)
('διατμήγω', 1)
('ἀτάλλω', 1)
('λείβω', 1)
('κτυπέω', 1)
('σπέρχω', 1)
('κρύπτω', 1)
('εἴλω', 1)
('ἧμαι', 1)
('θρῴσκω', 1)
('δηιόω', 1)
('ἵημι

In [29]:
import xml.etree.ElementTree as ET
import pandas as pd

# change FILENAME to local path
FILENAME = "tlg0012.tlg001.perseus-grc1.tb.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)
tree = ET.parse(LOCALPATH)
root = tree.getroot()

count=1         #sets counter keeping track of query results
prepid=10001    #in case the code encounters a prep first
wordhead=10000  #in case the code encounters a noun first
LEMMA = 'ὑπό'  #change as needed
CASE = 'a'      #change as needed
prepdict = {}
objlemma = 'tester'
nodeps = 0

for body in root.findall('./body'): 
    for sentence in body.findall('./sentence'):
        prepcount = 0
        matchlist = []
        for word in sentence.findall('./word'):
            if 'postag' in word.attrib:
                postag = word.get('postag')
                pos = postag[0]
                if (pos == 'r'):
                    preplemma = word.get('lemma')
                    if preplemma == LEMMA:
                        prepid = word.get('id')
                        prepcount += 1
                        
                        if prepid == wordhead:
                            matchlist.append('T')
                            if case == CASE:
                                objcount = prepdict.get(objlemma)
                                if (objcount == None):
                                    prepdict.update({objlemma: 1})
                                else:
                                    objcount += 1
                                    prepdict.update({objlemma: objcount})
                                #print(word.attrib)
                                #print(count) #counts query results
                                #count+=1
                        else:
                            matchlist.append('F')
                    #print(word.attrib)
                
                else: 
                    wordhead = word.get('head')
                    objlemma = word.get('lemma')
                    case = postag[7]
                    if prepid == wordhead:
                        matchlist.append('T')
                        if case == CASE:
                            objcount = prepdict.get(objlemma)
                            if (objcount == None):
                                prepdict.update({objlemma: 1})
                            else:
                                objcount += 1
                                prepdict.update({objlemma: objcount})
                            #print(word.attrib)
                            #print(count) #counts query results
                            #count+=1
                    else:
                        matchlist.append('F')
        
        numpreps = matchlist.count('T')
        if (numpreps != prepcount):
            extra = abs(numpreps - prepcount)
            nodeps += extra
        prepid = 10001
        wordhead = 10000

#list1 = prepdict.values()
#print(sum(list1))
sortedDict = sorted(prepdict.items(), key=lambda x: x[1], reverse=True)

df = pd.DataFrame(sortedDict)
#df.to_csv('prepobjs.csv', index=False)

print(*sortedDict, sep = '\n')
print("No dependent objs:", nodeps)

('Ἴλιος', 5)
('ζυγόν', 4)
('τεῖχος', 3)
('ὀστέον', 2)
('γαῖα', 2)
('ἀνθερεών', 1)
('χείρ', 1)
('λόφος', 1)
('μηνιθμός', 1)
('κλείς', 1)
('κῦμα', 1)
('πόλις', 1)
('ὄρος', 1)
('πούς', 1)
('ἄντυξ', 1)
('κρημνός', 1)
('νύξ', 1)
('πύργος', 1)
('λαπάρα', 1)
('ζόφος', 1)
('δειρή', 1)
('σπέος', 1)
('σάκος', 1)
No dependent objs: 63


In [ ]:
import xml.etree.ElementTree as ET
import networkx as nx

# change FILENAME to local path
FILENAME = "tlg0012.tlg001.perseus-grc1.tb.xml"
LOCALPATH  = os.path.join(LOCALROOT, 'gAGDT', 'data', 'xml', FILENAME)
tree = ET.parse(LOCALPATH)
root = tree.getroot()

G = nx.DiGraph()

prepid = 10001
preplemma = 10002
objhead = 10003
objlemma = 10004
objform = 10005
objcase = 10006


for sentence in root.findall(".//sentence"):
    for word in sentence.findall("./word"):
        if ('postag' in word.attrib):
            postag = word.get('postag')
            pos = postag[0]
            if (pos == 'r'):
                prepid = word.get('id')
                preplemma = word.get('lemma')
                if (objhead == prepid):
                    if G.has_node(objlemma):
                        if objcase == 'g':
                            G.nodes[objlemma].update({'gen': 0})
                            temp = G.nodes[objlemma]['gen']
                            temp += 1
                            G.nodes[objlemma].update(gen = temp)
                        if objcase == 'd':
                            G.nodes[objlemma].update({'dat': 0})
                            temp = G.nodes[objlemma]['dat']
                            temp += 1
                            G.nodes[objlemma].update(dat = temp)
                        if objcase == 'a':
                            G.nodes[objlemma].update({'acc': 0})
                            temp = G.nodes[objlemma]['acc']
                            temp += 1
                            G.nodes[objlemma].update(acc = temp)
                        if objcase == 'n':
                            G.nodes[objlemma].update({'nom': 0})
                            temp = G.nodes[objlemma]['nom']
                            temp += 1
                            G.nodes[objlemma].update(nom = temp)
                        if objcase == 'v':
                            G.nodes[objlemma].update({'voc': 0})
                            temp = G.nodes[objlemma]['voc']
                            temp += 1
                            G.nodes[objlemma].update(voc = temp)
                        temp = G.nodes[objlemma]['weight']
                        temp += 1
                        G.nodes[objlemma].update(weight = temp)
                    else:
                        G.add_node(objlemma, pos = pos, weight = 0)
                    
                    if G.has_edge(objlemma, preplemma):
                        G[objlemma][preplemma]['weight'] += 1
                    else:
                        G.add_edge(objlemma, preplemma, weight = 1)
                
                if G.has_node(preplemma):
                    if objcase == 'g':
                        G.nodes[preplemma].update({'gen': 0})
                        temp = G.nodes[preplemma]['gen']
                        temp += 1
                        G.nodes[preplemma].update(gen = temp)
                    if objcase == 'd':
                        G.nodes[preplemma].update({'dat': 0})
                        temp = G.nodes[preplemma]['dat']
                        temp += 1
                        G.nodes[preplemma].update(dat = temp)
                    if objcase == 'a':
                        G.nodes[preplemma].update({'acc': 0})
                        temp = G.nodes[preplemma]['acc']
                        temp += 1
                        G.nodes[preplemma].update(acc = temp)
                    if objcase == 'n':
                        G.nodes[preplemma].update({'nom': 0})
                        temp = G.nodes[preplemma]['nom']
                        temp += 1
                        G.nodes[preplemma].update(nom = temp)
                    if objcase == 'v':
                        G.nodes[preplemma].update({'voc': 0})
                        temp = G.nodes[preplemma]['voc']
                        temp += 1
                        G.nodes[preplemma].update(voc = temp)
                    temp = G.nodes[preplemma]['weight']
                    temp += 1
                    G.nodes[preplemma].update(weight = temp)
                else:
                    G.add_node(preplemma, pos = pos, weight = 0)

            elif (pos == 'n'):
                objhead = word.get('head')
                objlemma = word.get('lemma')
                objform = word.get('form')
                objpostag = word.get('postag')
                objcase = objpostag[7]
                if (objhead == prepid):
                    if G.has_node(objlemma):
                        if objcase == 'g':
                            G.nodes[objlemma].update('gen', 0)
                            temp = G.nodes[objlemma]['gen']
                            temp += 1
                            G.nodes[objlemma].update(gen = temp)
                        if objcase == 'd':
                            G.nodes[objlemma].update('dat', 0)
                            temp = G.nodes[objlemma]['dat']
                            temp += 1
                            G.nodes[objlemma].update(dat = temp)
                        if objcase == 'a':
                            G.nodes[objlemma].update('acc', 0)
                            temp = G.nodes[objlemma]['acc']
                            temp += 1
                            G.nodes[objlemma].update(acc = temp)
                        if objcase == 'n':
                            G.nodes[objlemma].update('nom', 0)
                            temp = G.nodes[objlemma]['nom']
                            temp += 1
                            G.nodes[objlemma].update(nom = temp)
                        if objcase == 'v':
                            G.nodes[objlemma].update('voc', 0)
                            temp = G.nodes[objlemma]['voc']
                            temp += 1
                            G.nodes[objlemma].update(voc = temp)
                        temp = G.nodes[objlemma]['weight']
                        temp += 1
                        G.nodes[objlemma].update(weight = temp)
                    else:
                        G.add_node(objlemma, pos = pos, weight = 0)
                    
                    if G.has_edge(objlemma, preplemma):
                        G[objlemma][preplemma]['weight'] += 1
                    else:
                        G.add_edge(objlemma, preplemma, weight = 1)
    prepid = 10001
    preplemma = 10002
    objhead = 10003
    objlemma = 10004
    objform = 10005
    objcase = 10006
    
#nx.write_graphml(G, "2021427prepNetwork.graphml")